In [1]:
data_var = '2024-02-11'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
13690,2024-02-11,Espanha Acb,08:30,Bilbao,Valencia,2.26,1.65,154.5,1.86,1.94,1.5,2.06,1.71,pfqqZeqi,0.442478,0.606061,0.537634,0.515464,0.048538,0.4,0.6,NaN,NaN,323.784,318.178220,0.982687,155.532,83.780301,0.538669,166.028,243.038,129.00,302.57,0.0,0.0,0.0,0.0,0.220632,0.029773,0.131293,-2.10,-0.420,-3.000000,0.0,0.0,0.0,4.39,0.878,0.740319,0.0,0.0,0.0
13691,2024-02-11,Espanha Acb,08:30,Breogan,Real Madrid,6.15,1.13,163.5,1.89,1.90,10.5,2.07,1.70,KCydWgT9,0.162602,0.884956,0.529101,0.526316,0.047557,1.0,0.6,NaN,NaN,254.946,106.002417,0.415784,113.626,38.085788,0.335186,149.340,121.288,142.29,110.88,0.0,0.0,0.0,0.0,0.975186,0.003731,0.138795,-0.20,-0.040,-128.750000,0.0,0.0,0.0,-0.68,-0.136,-0.955882,0.0,0.0,0.0
13692,2024-02-11,Espanha Acb,13:00,Baskonia,Joventut,1.29,3.66,167.5,1.82,1.94,-9.5,2.08,1.68,OprmYFbc,0.775194,0.273224,0.549451,0.515464,0.048418,0.8,0.4,NaN,NaN,152.282,32.264724,0.211875,172.614,53.807870,0.311724,130.562,212.218,107.69,188.34,0.0,0.0,0.0,0.0,0.677108,0.045134,0.150448,-0.13,-0.026,-11.153846,0.0,0.0,0.0,-0.91,-0.182,-14.615385,0.0,0.0,0.0
13693,2024-02-11,Espanha Acb,13:00,Murcia,Granada,1.18,5.05,159.5,1.80,1.96,-12.5,2.09,1.67,tMxhXZD3,0.847458,0.198020,0.555556,0.510204,0.045477,0.6,0.2,NaN,NaN,200.082,69.238591,0.346051,196.428,81.691494,0.415885,147.888,366.756,121.92,143.96,0.0,0.0,0.0,0.0,0.878492,0.060179,0.157971,0.81,0.162,1.111111,0.0,0.0,0.0,2.16,0.432,9.375000,0.0,0.0,0.0
13694,2024-02-11,Espanha Acb,14:30,Manresa,Barcelona,3.03,1.39,167.5,1.87,1.91,4.5,2.10,1.68,UZjr0xCa,0.330033,0.719424,0.534759,0.523560,0.049457,0.6,0.4,NaN,NaN,162.954,29.804535,0.182902,111.392,15.566616,0.139746,148.084,121.528,113.71,122.20,0.0,0.0,0.0,0.0,0.524731,0.014965,0.157135,-0.51,-0.102,-19.901961,0.0,0.0,0.0,-2.10,-0.420,-0.928571,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13830,2024-02-11,Áustria Liga Zweite,14:00,Basket Flames,Wagram,10.50,1.03,142.5,1.85,1.85,20.5,1.99,1.73,MomaerGQ,0.095238,0.970874,0.540541,0.540541,0.066112,0.6,0.0,NaN,NaN,327.610,123.460086,0.376851,391.778,675.475663,1.724129,312.794,0.000,228.69,81.00,0.0,0.0,0.0,0.0,1.161544,0.000000,0.098843,-2.69,-0.538,-17.657993,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
13831,2024-02-11,Bósnia E Herzegovina Prvenstvo Bih,14:00,Prijedor,KK Bosna,9.65,1.03,164.5,1.87,1.79,15.5,2.08,1.67,6VE6CoTn,0.103627,0.970874,0.534759,0.558659,0.074501,0.6,0.6,NaN,NaN,201.956,57.016622,0.282322,104.332,10.071389,0.096532,220.712,114.468,122.85,111.60,0.0,0.0,0.0,0.0,1.141435,0.030912,0.154621,0.29,0.058,149.137931,0.0,0.0,0.0,0.99,0.198,0.151515,0.0,0.0,0.0
13832,2024-02-11,Bósnia E Herzegovina Prvenstvo Bih,14:00,Promo,Slavija,1.44,2.65,157.5,1.85,1.85,-6.5,2.09,1.66,YsphgOSO,0.694444,0.377358,0.540541,0.540541,0.071803,0.6,0.4,NaN,NaN,276.280,157.854767,0.571358,231.252,103.235720,0.446421,274.996,455.766,477.90,355.84,1.0,0.0,0.0,0.0,0.418386,0.000000,0.162163,3.66,0.732,0.601093,0.0,0.0,0.0,-0.73,-0.146,-11.301370,0.0,0.0,0.0
13833,2024-02-11,Bósnia E Herzegovina Prvenstvo Bih Feminina,15:30,Leotar Trebinje F,Kozara F,1.12,5.61,130.5,1.83,1.83,-13.5,2.05,1.68,U5D5veVN,0.892857,0.178253,0.546448,0.546448,0.071110,0.0,0.0,NaN,NaN,155.354,72.768730,0.468406,148.714,53.934017,0.362669,0.000,0.000,80.50,86.

## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,00:00,Eua Nba,Portland Trail Blazers,New Orleans Pelicans,221.5,1.88,1.0000,Over
1,17:00,Eua Nba,Oklahoma City Thunder,Sacramento Kings,236.5,1.83,0.9999,Over
